# Setup

In [1]:
# !pip install -qU gradio diffusers transformers accelerate

Instalowane są cztery biblioteki:
- gradio - służy do tworzenia interfejsów graficznych dla modeli ML
- diffusers - zawiera implementacje różnych modeli generatywnych
- transformers - dostarcza modele językowe i narzędzia do ich obsługi
- accelerate - optymalizuje działanie modeli na różnych typach procesorów



In [ ]:
# Standard libraries
import os
# Scientific computing
import numpy as np
import torch
# Machine learning
from diffusers import StableDiffusionXLPipeline
# UI
import gradio as gr

## Biblioteki
* `numpy` - to fundament obliczeń numerycznych w Pythonie. W kontekście przetwarzania obrazów numpy jest niezbędny, ponieważ obrazy cyfrowe są reprezentowane jako wielowymiarowe tablice liczb.

* `PyTorch` (torch) - to framework uczenia maszynowego, który będzie odpowiedzialny za wykonywanie obliczeń na naszych modelach generatywnych.

* `diffusers` - `AutoencoderKL` to specjalny rodzaj sieci neuronowej, która potrafi kompresować i dekompresować obrazy, zachowując ich najważniejsze cechy. * `DiffusionPipeline` to ogólny interfejs do modeli generatywnych opartych na dyfuzji, a `StableDiffusionXLPipeline` to najnowsza, ulepszona wersja modelu Stable Diffusion, zaprojektowana do tworzenia obrazów o wysokiej jakości.

In [3]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [8]:
class CFG:
    model = "stabilityai/stable-diffusion-xl-base-1.0"
    device = 'cpu'
    dtype = torch.float16
    variant = "fp16"

if torch.backends.mps.is_available():
    CFG.device = "mps"
if torch.cuda.is_available():
    CFG.device = "cuda"



`model` wskazuje na konkretną wersję modelu Stable Diffusion XL - jest to "stabilityai/stable-diffusion-xl-base-1.0", czyli bazowy model SDXL w wersji 1.0 stworzony przez Stability AI. Ten model jest znany z generowania obrazów w wysokiej rozdzielczości.

`dtype` określa format danych jako torch.bfloat16 - jest to specjalny format liczb zmiennoprzecinkowych, który zapewnia dobry kompromis między precyzją obliczeń a zużyciem pamięci. Format bfloat16 wykorzystuje 16 bitów na liczbę, ale inaczej rozkłada bity między część całkowitą i ułamkową niż standardowy float16.

Ostatni parametr `variant` ustawiony na "fp16" informuje, że model będzie działał w trybie połowicznej precyzji (16-bitowej). Jest to zgodne z wcześniejszym ustawieniem dtype i pozwala na szybsze obliczenia przy akceptowalnej utracie precyzji.

# Funkcje

In [5]:
def infer(prompt, seed = 42, width=1024, height=1024,
          guidance_scale=5.0,  num_inference_steps= 5,
          progress=gr.Progress(track_tqdm=True)):

    generator = torch.Generator().manual_seed(seed)

    image = pipe(
        prompt = prompt, width = width, height = height,
        num_inference_steps = num_inference_steps,
        generator = generator, guidance_scale=guidance_scale
    ).images[0]

    return image, seed

Ta funkcja `infer` odpowiada za proces generowania obrazu na podstawie opisu tekstowego. Przeanalizujmy jej działanie krok po kroku.

Funkcja przyjmuje kilka parametrów wejściowych:
- `prompt` to opis tekstowy obrazu, który chcemy wygenerować
- `seed` (domyślnie 42) to ziarno losowości, które pozwala na odtworzenie dokładnie tego samego wyniku
- `width` i `height` (domyślnie 1024x1024) określają rozmiar generowanego obrazu w pikselach
- `guidance_scale` (domyślnie 5.0) kontroluje, jak bardzo model powinien trzymać się opisu tekstowego - wyższe wartości dają obrazy bardziej dosłownie odpowiadające promptowi
- `num_inference_steps` (domyślnie 5) określa liczbę kroków procesu denoising, czyli usuwania szumu z obrazu
- `progress` to pasek postępu z biblioteki gradio, który pokazuje stan generowania

W pierwszym kroku funkcja tworzy generator liczb losowych (`torch.Generator()`), którego ziarno jest ustawiane na podstawie parametru `seed`. Dzięki temu możemy później wygenerować dokładnie ten sam obraz, podając to samo ziarno.

Następnie wywoływany jest pipeline generacyjny (`pipe`), któremu przekazywane są wszystkie niezbędne parametry. Pipeline wykonuje właściwy proces generowania obrazu, który wewnętrznie polega na stopniowym usuwaniu szumu z losowo zainicjowanej macierzy pikseli, kierując się podanym opisem tekstowym.

Metoda zwraca krotkę zawierającą dwa elementy:
- **wygenerowany obraz** (pierwszy element z listy images zwróconej przez pipeline)
- **użyte ziarno losowości** (seed)

Ta implementacja pozwala na kontrolowane generowanie obrazów z możliwością powtórzenia dokładnie tego samego wyniku, co jest szczególnie przydatne przy debugowaniu i dopracowywaniu promptów.

# Aplikacja

In [9]:
pipe = StableDiffusionXLPipeline.from_pretrained(
      CFG.model,
      torch_dtype= CFG.dtype,
      variant = CFG.variant,
      use_safetensors=True).to(CFG.device)


MAX_SEED = np.iinfo(np.int32).max
MAX_IMAGE_SIZE = 2048

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.20steps/s]


W dalszej części kodu definiowane są dwa ważne ograniczenia systemu:
- `MAX_SEED` jest ustawiany na maksymalną wartość 32-bitowej liczby całkowitej (około 2.15 miliarda). Jest to górne ograniczenie dla ziarna losowości, które możemy użyć przy generowaniu obrazów.
- `MAX_IMAGE_SIZE` wynosi 2048 pikseli i określa maksymalny rozmiar obrazu, jaki może zostać wygenerowany. To ograniczenie wynika z limitów pamięci i mocy obliczeniowej dostępnego sprzętu.

Te stałe są istotne dla bezpiecznego działania systemu - zapobiegają próbom generowania zbyt dużych obrazów lub używania nieprawidłowych wartości ziarna losowości, co mogłoby prowadzić do błędów lub awarii systemu.

In [11]:
with gr.Blocks() as demo:

    with gr.Column(elem_id="col-container"):

        with gr.Row():

            prompt = gr.Text(
                label="Prompt", show_label=False,
                max_lines=1, placeholder="Enter your prompt",
                container=False,
            )
            run_button = gr.Button("Run", scale=0)

        result = gr.Image(label="Result", show_label=False)

        with gr.Accordion("Advanced Settings", open=False):

            seed = gr.Slider(
                label="Seed", minimum=0,
                maximum=MAX_SEED, step=1,
                value=0, )

            randomize_seed = gr.Checkbox(label="Randomize seed", value=True)

            with gr.Row():

                width = gr.Slider(
                    label="Width", minimum=256, maximum=MAX_IMAGE_SIZE,
                    step=32, value=512,
                )

                height = gr.Slider(
                    label="Height", minimum=256, maximum=MAX_IMAGE_SIZE,
                    step=32, value=1024,
                )

            with gr.Row():

                guidance_scale = gr.Slider(
                    label="Guidance Scale",
                    minimum=1, maximum=15, step=0.1, value=3.5,
                )

                num_inference_steps = gr.Slider(
                    label="Number of inference steps",
                    minimum=1, maximum=100, step=1, value= 5,
                )


    gr.on(
        triggers=[run_button.click, prompt.submit],
        fn = infer,
        inputs = [prompt, seed, width, height,
                  guidance_scale, num_inference_steps],
        outputs = [result, seed]
    )

demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9cbf4fb1d8823fbad0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/75 [00:00<?, ?steps/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9cbf4fb1d8823fbad0.gradio.live


`gr.Blocks()` zawiera całą strukturę interfejsu. Wewnątrz niego znajduje się kolumna (`gr.Column`), która organizuje wszystkie elementy pionowo.

W górnej części interfejsu znajduje się rząd (`gr.Row`) zawierający dwa kluczowe elementy: pole tekstowe do wprowadzania promptu oraz przycisk "Run". Pole tekstowe jest skonfigurowane jako jednoliniowe, z placeholder tekstem "Enter your prompt", bez wyświetlanej etykiety, co zapewnia czysty i minimalistyczny wygląd.

Pod tymi elementami znajduje się komponеnt wyświetlający wygenerowany obraz (`gr.Image`).

Sekcja "Advanced Settings" jest ukryta w rozwijanym panelu (`gr.Accordion`). Zawiera ona szereg kontrolek pozwalających na precyzyjne dostrojenie procesu generowania:

Pierwszy element to suwak do kontroli ziarna losowości (seed), który może przyjmować wartości od 0 do MAX_SEED. Obok niego znajduje się pole wyboru "Randomize seed", domyślnie zaznaczone, które pozwala na automatyczne losowanie ziarna.

Następnie mamy dwa suwaki w jednym rzędzie, kontrolujące szerokość i wysokość generowanego obrazu. Oba pozwalają na wybór wartości od 256 do `MAX_IMAGE_SIZE` pikseli, z krokiem co 32 piksele.

W ostatnim rzędzie znajdują się dwa istotne parametry:
- Guidance Scale (skala naprowadzania) - kontroluje, jak ściśle model ma trzymać się promptu
- Number of inference steps (liczba kroków wnioskowania) - określa, ile iteracji ma wykonać model podczas generowania obrazu

Na końcu kod konfiguruje obsługę zdarzeń - zarówno kliknięcie przycisku "Run", jak i wciśnięcie Enter w polu promptu wywołuje funkcję `infer`. Funkcja ta otrzymuje wszystkie wartości z kontrolek jako parametry wejściowe i zwraca wygenerowany obraz oraz użyte ziarno losowości.